In [ ]:
import pandas as pd
filename='sample_data_complaint.xlsx'
df=pd.read_excel(filename, usecols =["Complaint Text","Resolution"])
df.columns=["Complaint Text","Resolution"]
df


,Complaint Text,Resolution
0,A leakage was observed when using Alinity i-se...,A malfunction was identified regarding a leaka...
1,The customer indicated product shipment was re...,A malfunction was identified regarding ARCHITE...
2,Shipment of ARCHITECT Concentrated Wash Buffer...,A malfunction regarding Shipment of ARCHITECT ...
3,Alinity ci series System Software v2.6.2 on S...,A malfunction was identified for the Alinity c...
4,The customer observed bottles which appeared t...,No product deficiency and no malfunction was i...
...,...,...
95,The customer observed falsely elevated results...,A malfunction was identified regarding elevate...
96,The customer questioned higher than expected T...,A malfunction regarding higher Troponin result...
97,The customer reported insufficient fill volume...,A malfunction regarding insufficient fill volu...
98,The complaint text states that when customer r...,A malfunction was identified for the ARCHITECT...


In [ ]:
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

def clean_sentence(sentence,stopwords=False):
  sentence=sentence.lower().strip()
  sentence=re.sub(r'[^a-z0-9/s]',' ',sentence)

  if stopwords:
    sentence=remove_stopwords(sentence)

  return sentence

def get_cleaned_sentences(df,stopwords=False):
  sentences=df[['Complaint Text']]
  cleaned_sentences=[]

  for index,row in df.iterrows():
    cleaned=clean_sentence(row['Complaint Text'],stopwords)
    cleaned_sentences.append(cleaned)
  return cleaned_sentences
  
cleaned_sentences=get_cleaned_sentences(df,stopwords=True)
print(cleaned_sentences)

print("\n")

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords)


    

['leakage observed alinity series concentrated wash buffer', 'customer indicated product shipment received warm temperature stressed suitable use', 'shipment architect concentrated wash buffer list number 06c54 88 lot number 08381fn00 damaged transit fedex refused customer', 'alinity ci series software v2 6 2 scm01493 westgard rules aren t triggered versions assay installed instrument', 'customer observed bottles appeared distended architect concentrated wash buffer kit', 'customer observed activated read failures error code 1007 architect reaction vessels rv', 'customer observed microparticles bottle containing mold type deposits architect intact pth reagent', 'customer reported observing falsely elevated ldl hdl results 5 patients direct ldl reagent ln 1e31 20 ultra hdl reagent ln 3k33 21 architect 1601094 sum ldl hdl results patient greater total cholesterol results', 'customer observed multiple levels non abbott controls shifting high architect bnp calibrator', 'assay package inser

BAG OF WORDS MODEL

In [ ]:
import numpy as np

sentences=cleaned_sentences_with_stopwords

sentence_words=[[word for word in document.split()]for document in sentences]

from gensim import corpora
dictionary=corpora.Dictionary(sentence_words)
for key,value in dictionary.items():
  print(key," : ",value)

import pprint

bow_corpus=[dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow_corpus):
  print(sent)
  print(embedding)

question_orig="product shipment was received warm and was not suitable for use"
question=clean_sentence(question_orig,stopwords=False)
question_embedding=dictionary.doc2bow(question.split())

print("\n\n",question,"\n",question_embedding)

0  :  a
1  :  alinity
2  :  buffer
3  :  concentrated
4  :  i
5  :  leakage
6  :  observed
7  :  series
8  :  using
9  :  was
10  :  wash
11  :  when
12  :  and
13  :  customer
14  :  for
15  :  indicated
16  :  not
17  :  product
18  :  received
19  :  shipment
20  :  stressed
21  :  suitable
22  :  temperature
23  :  the
24  :  therefore
25  :  use
26  :  warm
27  :  06c54
28  :  08381fn00
29  :  88
30  :  architect
31  :  by
32  :  damaged
33  :  fedex
34  :  in
35  :  list
36  :  lot
37  :  number
38  :  of
39  :  refused
40  :  transit
41  :  2
42  :  6
43  :  are
44  :  aren
45  :  assay
46  :  ci
47  :  installed
48  :  instrument
49  :  on
50  :  rules
51  :  same
52  :  scm01493
53  :  software
54  :  system
55  :  t
56  :  there
57  :  triggered
58  :  two
59  :  v2
60  :  versions
61  :  westgard
62  :  where
63  :  while
64  :  appeared
65  :  be
66  :  bottles
67  :  distended
68  :  kit
69  :  to
70  :  which
71  :  1007
72  :  activated
73  :  code
74  :  error
75  :  fa

COSINE SIMILARITY


In [ ]:
import sklearn 
from sklearn.metrics.pairwise import cosine_similarity

def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
  max_sim=-1
  index_sim=-1
  for index,faq_embedding in enumerate(sentence_embeddings):
    sim=cosine_similarity(faq_embedding,question_embedding)[0][0]
    print(index,sim,sentences[index])
    if sim>max_sim:
      max_sim=sim
      index_sim=index
  
  print("\n")
  print("question : ",question)
  print("\n")
  print("Retrieved : ",FAQdf.iloc[index_sim,0])
  print("\n")
  print(FAQdf.iloc[index_sim,1])

retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df,sentences)

0 0.21693045781865616 a leakage was observed when using alinity i series concentrated wash buffer 
1 0.9941724026247171 the customer indicated product shipment was received warm  temperature stressed  and therefore not suitable for use
2 0.970142500145332 shipment of architect concentrated wash buffer  list number 06c54 88  lot number 08381fn00  was damaged in transit by fedex and refused by the customer
3 0.8436614877321074 alinity ci series system software v2 6 2 on scm01493 where the westgard rules aren t triggered while there are two versions of the same assay installed on the instrument 
4 0.970142500145332 the customer observed bottles which appeared to be distended when using architect concentrated wash buffer kit
5 0.9985681322700889 the customer observed activated read failures  error code 1007   when using architect reaction vessels  rv 
6 0.21693045781865616 the customer observed a microparticles bottle containing   mold type deposits    while using architect intact pth reag

GLOVE EMBEDDING

In [ ]:
from gensim.models import word2vec
import gensim.downloader as api

glove_model=None
try:
  glove_model=gensim.models.KeyedVectors.load("./glovemodel.mod")
  print("loaded glove model")

except:
  glove_model=api.load("glove-twitter-25")
  glove_model.save("./glovemodel.mod")
  print("saved glove model")

v2w_model=None
try:
  v2w_model=gensim.models.KeyedVectors.load("./w2vecmodel.mod")
  print("loaded w2v model")

except:
  v2w_model=api.load("word2vec-google-news-300")
  v2w_model.save("./w2vecmodel.mod")
  print("saved w2v model")

w2vec_embedding_size=len(v2w_model['computer'])
glove_embedding_size=len(glove_model['computer'])

FIXING NEW WORDS THAT ARE NOT IN THE DICTIONARY

In [ ]:
def getWordVec(word,model):
  samp=model['computer']
  vec=[0]*len(samp)
  try:
    vec=model[word]
  except:
    vec=[0]*len(samp)
  return (vec)

def getPhraseEmbedding(phrase,embeddingmodel):
  samp=getWordVec('computer',embeddingmodel)
  vec=np.array([0]*len(samp))
  den=0
  for word in phrase.split():
    den=den+1
    vec=vec+np.array(getWordVec(word,embeddingmodel))

  return vec.reshape(1,-1)

WITH W2VEC

In [ ]:
sent_embeddings=[]
for sent in cleaned_sentences:
  sent_embeddings.append(getPhraseEmbedding(sent,v2w_model))

question_embedding=getPhraseEmbedding(question,v2w_model)
retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df,cleaned_sentences)

0 0.34012017205464523 leakage observed alinity series concentrated wash buffer
1 0.7974641908005894 customer indicated product shipment received warm temperature stressed suitable use
2 0.545219782014277 shipment architect concentrated wash buffer list number 06c54 88 lot number 08381fn00 damaged transit fedex refused customer
3 0.34787633583491906 alinity ci series software v2 6 2 scm01493 westgard rules aren t triggered versions assay installed instrument
4 0.4312455821152203 customer observed bottles appeared distended architect concentrated wash buffer kit
5 0.42240744782387707 customer observed activated read failures error code 1007 architect reaction vessels rv
6 0.40230690079681913 customer observed microparticles bottle containing mold type deposits architect intact pth reagent
7 0.3512663053895849 customer reported observing falsely elevated ldl hdl results 5 patients direct ldl reagent ln 1e31 20 ultra hdl reagent ln 3k33 21 architect 1601094 sum ldl hdl results patient grea

In [ ]:
pip install ktrain

     |████████████████████████████████| 25.3MB 1.5MB/s 
     |████████████████████████████████| 6.8MB 49.8MB/s 
     |████████████████████████████████| 983kB 68.4MB/s 
     |████████████████████████████████| 266kB 64.2MB/s 
     |████████████████████████████████| 1.3MB 41.4MB/s 
     |████████████████████████████████| 1.2MB 25.0MB/s 
     |████████████████████████████████| 471kB 38.8MB/s 
     |████████████████████████████████| 890kB 41.5MB/s 
     |████████████████████████████████| 2.9MB 49.3MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.4-cp36-none-any.whl size=25276443 sha256=24931f9ff7c62be82cc55aeb52eefff07629949bfb9e21fea037567afe249334
  Stored in directory: /root/.cache/pip/wheels/1b/77/8a/bdceaabc308e7178d575278bf6143b7d1a9b939a1e40c56b88
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=62f81d33fd856cf865672ad0b3ae419ebd86cfb6b79824307e5bc70522ebf3ed
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

BERT EMBEDDINGS

The issue is the server setting part where the bert_serving_client is not executing.

In [ ]:
from bert_serving.client import BertClient
bc=BertClient(ip='localhost')
res=bc.encode(['ml','ai'])
print(res)

KeyboardInterrupt: ignored

In [ ]:
question=clean_sentence(question,orig,stopwords=False)
cleaned_snetences=get_cleaned_sentences(df,stopwords=False)

sent_bertphrase_embeddings=[]
for sent in cleaned_sentences:
  sent_bertphrase_embeddings.append(bc.encode([sent]))

question_embedding=bc.encode([question])
retrieveAndPrintFAQAnswer(question_embedding,sent_bertphrase_embeddings,df,cleaned_sentences)

In [ ]:
pip install bert-serving-server

ERROR: Operation cancelled by user


In [ ]:
pip install bert-serving-client

In [ ]:
pip install bert-embedding

     |████████████████████████████████| 13.8MB 294kB/s 
     |████████████████████████████████| 29.6MB 1.4MB/s 
     |████████████████████████████████| 215kB 54.9MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-cp36-none-any.whl size=259917 sha256=2a0973a486503349a546449236673a92d2c6d8fc7ada86a82990708090bd6ef8
  Stored in directory: /root/.cache/pip/wheels/ff/48/ac/a77c79aa416ba6dd7bf487f2280b0471034f66141617965914
Successfully built gluonnlp
ERROR: xarray 0.15.1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.14.6 which is incompatible.
ERROR: tifffile 2020.9.3 has requirement numpy>=1.15.1, but you'll have numpy 1.14.6 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement numpy~=1.19.2, but you'll have numpy 1.14.6 which is incompatible.
ERROR: spacy 2.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: seaborn 0.